## Applying code to the Rutford Ice Stream Data

In [1]:
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy import read

import matplotlib.pyplot as plt
# import ipynb.fs 
import icequake_processing
import importlib
importlib.reload(icequake_processing)
from os.path import exists

In [3]:
filename = "YG.ST01.2009-01-03.mseed"

# if exists(filename):
#     print ("File exists, loading it.")
#     st_raw = read(filename)
# else:
# print ("File doesn't exist, downloading it.")
client = Client("IRIS")

t1 = UTCDateTime("2009-01-03T00:00:00")
t2 = UTCDateTime("2009-01-04T00:00:00")
# t1 = UTCDateTime("2009-01-03T05:00:00")
# t2 = UTCDateTime("2009-01-03T06:00:00")
#http://ds.iris.edu/mda/YG/?starttime=2009-01-01&endtime=2009-02-03

# This period seems to have tremor?
# t1 = UTCDateTime("2009-01-10T00:00:00")
# t2 = UTCDateTime("2009-01-10T01:00:00")

# Event from the paper
# t1 = UTCDateTime("2009-01-03T05:15:42")
# t2 = UTCDateTime("2009-01-03T05:15:44")

st_raw = client.get_waveforms("YG", "ST01", "--", "GL1", t1, t2, attach_response=True)

inv = client.get_stations(network="YG", station="ST01",
                                 starttime=t1,
                                endtime=t2, level="response")
st_raw.write(filename, format="MSEED")  

In [4]:
st_day = st_raw.copy()
pre_filt = (5, 10, 250, 500)
st_day.remove_response(output='VEL', pre_filt=pre_filt)

1 Trace(s) in Stream:
YG.ST01..GL1 | 2009-01-03T00:00:00.000000Z - 2009-01-04T00:00:00.000000Z | 1000.0 Hz, 86400001 samples

In [ ]:
length = (st[0].stats.endtime - st[0].stats.starttime)
new_time = (st[0].stats.starttime + length/2)
print(new_time)

In [ ]:
importlib.reload(icequake_processing)

st = st_day
avg_power,xf = icequake_processing.better_ppsd(st, new_time, 
                                         number_of_one_hour_intervals = 24, 
                                         filename = "full_day_ris_ppsd")

In [ ]:
fig,ax=plt.subplots()
fig.patch.set_facecolor('white')
plt.plot(xf,avg_power)
plt.yscale('log')
plt.xscale('log')
plt.show()

In [ ]:
from obspy.signal.trigger import classic_sta_lta
from obspy.signal.trigger import plot_trigger
from obspy.signal.trigger import trigger_onset

tr=st[0]
df = tr.stats.sampling_rate
cft = classic_sta_lta(tr, 0.01*df, 2*df)
# cft = classic_sta_lta(tr, 1*df, 100*df)

In [ ]:
# trig_on = 150
# trig_off = 75
# plot_trigger(tr,cft,trig_on,trig_off)
# event_times = trigger_onset(cft,trig_on,trig_off)/df
# len(event_times)

In [ ]:
st = st_day

In [ ]:
st.plot(starttime=UTCDateTime("2009-01-03T05:19:08.7"), endtime=UTCDateTime("2009-01-03T05:19:08.8"))

In [ ]:
tr.data



In [ ]:
from scipy.fft import fft, fftfreq
import numpy as np

tr = st.select(id="YG.ST01..GL1")[0]
N = len(tr.data)
T = tr.stats.delta
yf = fft(tr.data)
xf = fftfreq(N, T)[:N//2]
ya = 2.0/N * np.abs(yf[0:N//2])
import matplotlib.pyplot as plt
plt.plot(xf, ya**2)
plt.grid()

# plt.yscale("log")
plt.xscale("log")
plt.xlim([1,1000])
# plt.ylim([1e-14,1e-10])

plt.show()